In [2]:
import numpy as np
import pandas as pd
import PIL
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,LocallyConnected2D,BatchNormalization,AveragePooling2D
from tensorflow.keras.preprocessing import image

In [11]:
nRowsRead = 20000 # specify 'None' if want to read whole file
# list_bbox_celeba.csv has 202599 rows in reality, but we are only loading/previewing the first 1000 rows
df = pd.read_csv('list_landmarks_align_celeba.csv', delimiter=',')
df.dataframeName = 'landmarks'
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')
print(df["lefteye_x"].std())
print(df["lefteye_y"].std())
print(df["righteye_x"].std())
print(df["righteye_y"].std())

There are 202599 rows and 11 columns
1.7179517169083085
1.1292841771227071
1.6902521943883553
1.1692294395933214


In [3]:
df.columns
df = df[60000:80000]

In [4]:
images_data_path = "C:/Users/foreh/Documents/celebaRepo/FacialAttrCNN/celeba/greyscale"

In [5]:
width = 178
height = 218
image_size = (width,height)
images_data = list()
for idx in range(df.shape[0]):
    path = "{}/{}".format(str(images_data_path),str(df.iloc[idx].image_id))
    image = PIL.Image.open(path)
    image_array = np.asarray(image)
    images_data.append(image_array)
images_data = np.array(images_data)

In [6]:
def image_from_index(index, size=(image_size), path=images_data_path):
        path = "{}/{}".format(str(path),str(df.iloc[index].image_id))
        # to read the image
        image = PIL.Image.open(path).resize(size)
        image_array = np.asarray(image)

In [7]:
images = images_data # list of array of images
labels = df[["image_id", "lefteye_y","righteye_y"]]#, 'nose_x', 'nose_y', 'leftmouth_x', 'leftmouth_y', 'rightmouth_x','rightmouth_y']]
train_images,test_images,train_labels,test_labels = train_test_split(images,labels,test_size=0.3,random_state=(6))
y_test=test_labels.drop(["image_id"],axis=1)
y_train=train_labels.drop(["image_id"],axis=1)
X_test=test_images
X_train=train_images
#X_train = tf.expand_dims(X_train, axis=-1)
print("Samples Training Set:",len(X_train))
X_test, X_val, y_test, y_val=train_test_split(X_test, y_test,test_size=0.5,random_state=(5))
print("Samples in Validation Data:",len(X_val))
print("Samples in Test Data:",len(X_test))

Samples Training Set: 14000
Samples in Validation Data: 3000
Samples in Test Data: 3000


In [8]:
model2 = Sequential()
model2.add(tf.keras.Input(shape=(image_size[1],image_size[0], 1)))
model2.add(MaxPooling2D(2))
model2.add(Conv2D(12, (3,3), strides=(3,3),padding="same"))
model2.add(Conv2D(8, (3,3), strides=(2,1)))
model2.add(Dropout(.2))
model2.add(BatchNormalization())
model2.add(Conv2D(16, (2,1)))
model2.add(MaxPooling2D(2))
model2.add(BatchNormalization())
#model2.add(Conv2D(8, (2,2)))
#model2.add(Conv2D(16, (3,3)))
model2.add(MaxPooling2D(2))
model2.add(Flatten())
model2.add(Dense(44))
model2.add(Dense(32))
model2.add(Dense(2, activation="linear"))
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 max_pooling2d (MaxPooling2D  (None, 109, 89, 1)       0         
 )                                                               
                                                                 
 conv2d (Conv2D)             (None, 37, 30, 12)        120       
                                                                 
 conv2d_1 (Conv2D)           (None, 18, 28, 8)         872       
                                                                 
 dropout (Dropout)           (None, 18, 28, 8)         0         
                                                                 
 batch_normalization (BatchN  (None, 18, 28, 8)        32        
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 17, 28, 16)        2

In [9]:
#model2 = tf.keras.models.load_model("eyes_loc/iter2")

In [10]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001)

model2.compile(optimizer = "sgd",loss="mean_squared_error",metrics = ['acc', 'mse'])

In [12]:
training_process = model2.fit(X_train,y_train,epochs = 200,validation_data = (X_val,y_val),batch_size=64,shuffle = True)

Epoch 1/200
219/219 [==============================] - 2s 10ms/step - loss: 7.8344 - acc: 0.5357 - mse: 7.8344 - val_loss: 8491.4844 - val_acc: 0.6640 - val_mse: 8491.4844
Epoch 2/200
219/219 [==============================] - 2s 9ms/step - loss: 8.7768 - acc: 0.5284 - mse: 8.7768 - val_loss: 9431.9023 - val_acc: 0.6517 - val_mse: 9431.9023
Epoch 3/200
219/219 [==============================] - 2s 9ms/step - loss: 7.0816 - acc: 0.5429 - mse: 7.0816 - val_loss: 7512.6958 - val_acc: 0.6550 - val_mse: 7512.6958
Epoch 4/200
219/219 [==============================] - 2s 9ms/step - loss: 8.3609 - acc: 0.5262 - mse: 8.3609 - val_loss: 7490.2593 - val_acc: 0.6633 - val_mse: 7490.2593
Epoch 5/200
219/219 [==============================] - 2s 10ms/step - loss: 7.3414 - acc: 0.5274 - mse: 7.3414 - val_loss: 7559.3960 - val_acc: 0.5297 - val_mse: 7559.3960
Epoch 6/200
219/219 [==============================] - 2s 9ms/step - loss: 7.3039 - acc: 0.5299 - mse: 7.3039 - val_loss: 7235.8672 - val_acc: 

Epoch 96/200
219/219 [==============================] - 2s 10ms/step - loss: 1.6680 - acc: 0.5554 - mse: 1.6680 - val_loss: 69.7489 - val_acc: 0.3513 - val_mse: 69.7489
Epoch 97/200
219/219 [==============================] - 2s 10ms/step - loss: 1.6929 - acc: 0.5549 - mse: 1.6929 - val_loss: 50.8026 - val_acc: 0.6523 - val_mse: 50.8026
Epoch 98/200
219/219 [==============================] - 2s 10ms/step - loss: 1.6075 - acc: 0.5490 - mse: 1.6075 - val_loss: 65.5050 - val_acc: 0.3473 - val_mse: 65.5050
Epoch 99/200
219/219 [==============================] - 2s 10ms/step - loss: 1.6847 - acc: 0.5484 - mse: 1.6847 - val_loss: 252.9966 - val_acc: 0.6557 - val_mse: 252.9966
Epoch 100/200
219/219 [==============================] - 2s 11ms/step - loss: 1.6956 - acc: 0.5486 - mse: 1.6956 - val_loss: 43.9254 - val_acc: 0.6623 - val_mse: 43.9254
Epoch 101/200
219/219 [==============================] - 2s 10ms/step - loss: 1.5422 - acc: 0.5508 - mse: 1.5422 - val_loss: 39.0869 - val_acc: 0.6600 -

Epoch 193/200
219/219 [==============================] - 3s 12ms/step - loss: 1.0644 - acc: 0.6631 - mse: 1.0644 - val_loss: 1.0794 - val_acc: 0.6817 - val_mse: 1.0794
Epoch 194/200
219/219 [==============================] - 3s 12ms/step - loss: 1.0292 - acc: 0.6721 - mse: 1.0292 - val_loss: 0.9653 - val_acc: 0.6810 - val_mse: 0.9653
Epoch 195/200
219/219 [==============================] - 3s 12ms/step - loss: 0.9925 - acc: 0.6755 - mse: 0.9925 - val_loss: 3.7852 - val_acc: 0.7413 - val_mse: 3.7852
Epoch 196/200
219/219 [==============================] - 3s 12ms/step - loss: 1.0320 - acc: 0.6749 - mse: 1.0320 - val_loss: 3.0748 - val_acc: 0.7400 - val_mse: 3.0748
Epoch 197/200
219/219 [==============================] - 3s 12ms/step - loss: 1.0278 - acc: 0.6669 - mse: 1.0278 - val_loss: 1.4341 - val_acc: 0.7230 - val_mse: 1.4341
Epoch 198/200
219/219 [==============================] - 3s 12ms/step - loss: 1.0024 - acc: 0.6698 - mse: 1.0024 - val_loss: 5.1742 - val_acc: 0.7130 - val_mse:

In [13]:
model2.evaluate(X_val, y_val)

94/94 [==============================] - 1s 5ms/step - loss: 1.7837 - acc: 0.7013 - mse: 1.7837


[1.7837073802947998, 0.7013333439826965, 1.7837073802947998]

In [14]:
model2.save("eyes_loc/eyesy")

INFO:tensorflow:Assets written to: eyes_loc/eyesy\assets


history = model2.fit_generator(train_generator, steps_per_epoch = 100, epochs = 30,
validation_data = validation_generator, validation_steps = 50)

In [14]:
images = images_data # list of array of images
labels = df[["image_id", "width", "height"]]  # dataframe of image features
train_images,test_images,train_labels,test_labels = train_test_split(images,labels,test_size=0.3,random_state=(6))
y_test=test_labels.drop(["image_id"],axis=1)
y_train=train_labels.drop(["image_id"],axis=1)
X_test=test_images
X_train=train_images
#X_train = tf.expand_dims(X_train, axis=-1)
print("Samples Training Set:",len(X_train))
X_test, X_val, y_test, y_val=train_test_split(X_test, y_test,test_size=0.5,random_state=(5))
print("Samples in Validation Data:",len(X_val))
print("Samples in Test Data:",len(X_test))

KeyError: "['width', 'height'] not in index"

In [ ]:
model = Sequential()
model.add(tf.keras.Input(shape=(image_size[1],image_size[0], 1)))
model.add(Conv2D(24, (3,3), strides=(3,3),padding="valid"))
model.add(Conv2D(8, (3,3), strides=(2,1)))
model.add(MaxPooling2D(2))
model.add(Dropout(.2))
model.add(BatchNormalization())
model.add(Conv2D(16, (3,3)))
model.add(MaxPooling2D(2))
model.add(BatchNormalization())
model.add(Conv2D(8, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(MaxPooling2D(2))
model.add(Flatten())
model.add(Dense(44))
model.add(Dense(32))
model.add(Dense(2, activation="linear"))
model.summary()

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(optimizer = opt,loss="huber_loss",metrics = ['acc', 'mse'])

In [ ]:
training_process = model.fit(X_train,y_train,epochs = 10,validation_data = (X_val,y_val),batch_size=8,shuffle = True)

In [ ]:
#model.save("faceSizeRegression/iter2") #iter1: .9993 accuracy
model3 = tf.keras.models.load_model("faceSizeRegression/iter1")
#model3.evaluate(X_val,y_val)
img = image_from_index(999)
tf.keras.training.predict(model3, img)